In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from scipy import stats

In [2]:
data = pd.read_csv("german-data-numeric-formatted.csv")

In [3]:
data.head(10)

,Column1,Column2,Column3,Column4,Column5,Column6,Column7,Column8,Column9,Column10,...,Column18,Column19,Column20,Column21,Column22,Column23,Column24,Column25,Column26,Column27
0,NaN,1,6,4,12,5,5,3,4,1,...,0,1,0,0,1,0,0,1,1,NaN
1,NaN,2,48,2,60,1,3,2,2,1,...,0,1,0,0,1,0,0,1,2,NaN
2,NaN,4,12,4,21,1,4,3,3,1,...,0,1,0,0,1,0,1,0,1,NaN
3,NaN,1,42,2,79,1,4,3,4,2,...,0,0,0,0,0,0,0,1,1,NaN
4,NaN,1,24,3,49,1,3,3,4,4,...,0,1,0,0,0,0,0,1,2,NaN
5,NaN,4,36,2,91,5,3,3,4,4,...,0,1,0,0,0,0,1,0,1,NaN
6,NaN,4,24,2,28,3,5,3,4,2,...,0,1,0,0,1,0,0,1,1,NaN
7,NaN,2,36,2,69,1,3,3,2,3,...,1,1,0,1,0,0,0,0,1,NaN
8,NaN,4,12,2,31,4,4,1,4,1,...,0,1,0,0,1,0,1,0,1,NaN
9,NaN,2,30,4,52,1,1,4,2,3,...,0,1,0,0,1,0,0,0,2,NaN


In [4]:
data.shape

(1000, 27)

In [5]:
data['Column1'].isnull().sum()

1000

In [6]:
data['Column27'].isnull().sum()

1000

In [7]:
data = data.drop(['Column1', 'Column27'], axis = 1)

In [8]:
data

,Column2,Column3,Column4,Column5,Column6,Column7,Column8,Column9,Column10,Column11,...,Column17,Column18,Column19,Column20,Column21,Column22,Column23,Column24,Column25,Column26
0,1,6,4,12,5,5,3,4,1,67,...,0,0,1,0,0,1,0,0,1,1
1,2,48,2,60,1,3,2,2,1,22,...,0,0,1,0,0,1,0,0,1,2
2,4,12,4,21,1,4,3,3,1,49,...,0,0,1,0,0,1,0,1,0,1
3,1,42,2,79,1,4,3,4,2,45,...,0,0,0,0,0,0,0,0,1,1
4,1,24,3,49,1,3,3,4,4,53,...,1,0,1,0,0,0,0,0,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,4,12,2,17,1,4,2,4,1,31,...,0,0,1,0,0,1,0,1,0,1
996,1,30,2,39,1,3,1,4,2,40,...,0,1,1,0,0,1,0,0,0,1
997,4,12,2,8,1,5,3,4,3,38,...,0,0,1,0,0,1,0,0,1,1
998,1,45,2,18,1,3,3,4,4,23,...,0,0,1,0,0,0,0,0,1,2


In [9]:
data.columns = ['Checking Account','Duration (months)', 'Credit History', 
                'Column5 - Credit Amount?', 'Savings Account/Bonds', 
                'Employed Since', 'Personal Status/Sex', 'Resident Since', 
                'Property', 'Age(years)', 'Other Installment Plans', 
                'Number of Existing Credits', 'Number of liable people', 
                'Telephone', 'Foreign Worker', 'Column17', 'Column18', 
                'Column19', 'Column20', 'Column21', 'Column22', 'Column23', 
                'Column24', 'Column25', 'Cost Matrix']

In [10]:
data

,Checking Account,Duration (months),Credit History,Column5 - Credit Amount?,Savings Account/Bonds,Employed Since,Personal Status/Sex,Resident Since,Property,Age(years),...,Column17,Column18,Column19,Column20,Column21,Column22,Column23,Column24,Column25,Cost Matrix
0,1,6,4,12,5,5,3,4,1,67,...,0,0,1,0,0,1,0,0,1,1
1,2,48,2,60,1,3,2,2,1,22,...,0,0,1,0,0,1,0,0,1,2
2,4,12,4,21,1,4,3,3,1,49,...,0,0,1,0,0,1,0,1,0,1
3,1,42,2,79,1,4,3,4,2,45,...,0,0,0,0,0,0,0,0,1,1
4,1,24,3,49,1,3,3,4,4,53,...,1,0,1,0,0,0,0,0,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,4,12,2,17,1,4,2,4,1,31,...,0,0,1,0,0,1,0,1,0,1
996,1,30,2,39,1,3,1,4,2,40,...,0,1,1,0,0,1,0,0,0,1
997,4,12,2,8,1,5,3,4,3,38,...,0,0,1,0,0,1,0,0,1,1
998,1,45,2,18,1,3,3,4,4,23,...,0,0,1,0,0,0,0,0,1,2


In [11]:
data = pd.read_csv("german-formatted.csv")

In [12]:
data

,Column1,Column2,Column3,Column4,Column5,Column6,Column7,Column8,Column9,Column10,...,Column12,Column13,Column14,Column15,Column16,Column17,Column18,Column19,Column20,Column21
0,A11,6,A34,A43,1169,A65,A75,4,A93,A101,...,A121,67,A143,A152,2,A173,1,A192,A201,1
1,A12,48,A32,A43,5951,A61,A73,2,A92,A101,...,A121,22,A143,A152,1,A173,1,A191,A201,2
2,A14,12,A34,A46,2096,A61,A74,2,A93,A101,...,A121,49,A143,A152,1,A172,2,A191,A201,1
3,A11,42,A32,A42,7882,A61,A74,2,A93,A103,...,A122,45,A143,A153,1,A173,2,A191,A201,1
4,A11,24,A33,A40,4870,A61,A73,3,A93,A101,...,A124,53,A143,A153,2,A173,2,A191,A201,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,A14,12,A32,A42,1736,A61,A74,3,A92,A101,...,A121,31,A143,A152,1,A172,1,A191,A201,1
996,A11,30,A32,A41,3857,A61,A73,4,A91,A101,...,A122,40,A143,A152,1,A174,1,A192,A201,1
997,A14,12,A32,A43,804,A61,A75,4,A93,A101,...,A123,38,A143,A152,1,A173,1,A191,A201,1
998,A11,45,A32,A43,1845,A61,A73,4,A93,A101,...,A124,23,A143,A153,1,A173,1,A192,A201,2


In [13]:
new_names={'Column1':'Status Existing Checking Account','Column2':'Duration in month','Column3':'Credit History','Column4':'Purpose','Column5':'Credit_amount','Column6':'Savings account/bonds','Column7':'Present Employment Since','Column8':'Installment rate in percentage of disposable income','Column9':'Personal status and sex','Column10':'Other debtors/guarantors','Column11':'Present residence since','Column12':'Property','Column13':'Age in years','Column14':'Other installment plans','Column15':'Housing','Column16':'Number of existing credits at this bank','Column17':'Job','Column18':'Number of people being liable to provide maintenance for','Column19':'Telephone','Column20':'Foreign worker','Column21':'Cost Matrix'}
data.rename(columns=new_names, inplace=True)

In [14]:
data

,Status Existing Checking Account,Duration in month,Credit History,Purpose,Credit_amount,Savings account/bonds,Present Employment Since,Installment rate in percentage of disposable income,Personal status and sex,Other debtors/guarantors,...,Property,Age in years,Other installment plans,Housing,Number of existing credits at this bank,Job,Number of people being liable to provide maintenance for,Telephone,Foreign worker,Cost Matrix
0,A11,6,A34,A43,1169,A65,A75,4,A93,A101,...,A121,67,A143,A152,2,A173,1,A192,A201,1
1,A12,48,A32,A43,5951,A61,A73,2,A92,A101,...,A121,22,A143,A152,1,A173,1,A191,A201,2
2,A14,12,A34,A46,2096,A61,A74,2,A93,A101,...,A121,49,A143,A152,1,A172,2,A191,A201,1
3,A11,42,A32,A42,7882,A61,A74,2,A93,A103,...,A122,45,A143,A153,1,A173,2,A191,A201,1
4,A11,24,A33,A40,4870,A61,A73,3,A93,A101,...,A124,53,A143,A153,2,A173,2,A191,A201,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,A14,12,A32,A42,1736,A61,A74,3,A92,A101,...,A121,31,A143,A152,1,A172,1,A191,A201,1
996,A11,30,A32,A41,3857,A61,A73,4,A91,A101,...,A122,40,A143,A152,1,A174,1,A192,A201,1
997,A14,12,A32,A43,804,A61,A75,4,A93,A101,...,A123,38,A143,A152,1,A173,1,A191,A201,1
998,A11,45,A32,A43,1845,A61,A73,4,A93,A101,...,A124,23,A143,A153,1,A173,1,A192,A201,2


In [15]:
df_one_hot_encoded = pd.get_dummies(data = data, columns = ["Status Existing Checking Account"], prefix = "CheckingAccount")

df_one_hot_encoded

,Duration in month,Credit History,Purpose,Credit_amount,Savings account/bonds,Present Employment Since,Installment rate in percentage of disposable income,Personal status and sex,Other debtors/guarantors,Present residence since,...,Number of existing credits at this bank,Job,Number of people being liable to provide maintenance for,Telephone,Foreign worker,Cost Matrix,CheckingAccount_A11,CheckingAccount_A12,CheckingAccount_A13,CheckingAccount_A14
0,6,A34,A43,1169,A65,A75,4,A93,A101,4,...,2,A173,1,A192,A201,1,1,0,0,0
1,48,A32,A43,5951,A61,A73,2,A92,A101,2,...,1,A173,1,A191,A201,2,0,1,0,0
2,12,A34,A46,2096,A61,A74,2,A93,A101,3,...,1,A172,2,A191,A201,1,0,0,0,1
3,42,A32,A42,7882,A61,A74,2,A93,A103,4,...,1,A173,2,A191,A201,1,1,0,0,0
4,24,A33,A40,4870,A61,A73,3,A93,A101,4,...,2,A173,2,A191,A201,2,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,12,A32,A42,1736,A61,A74,3,A92,A101,4,...,1,A172,1,A191,A201,1,0,0,0,1
996,30,A32,A41,3857,A61,A73,4,A91,A101,4,...,1,A174,1,A192,A201,1,1,0,0,0
997,12,A32,A43,804,A61,A75,4,A93,A101,4,...,1,A173,1,A191,A201,1,0,0,0,1
998,45,A32,A43,1845,A61,A73,4,A93,A101,4,...,1,A173,1,A192,A201,2,1,0,0,0


In [16]:
df_one_hot_encoded = pd.get_dummies(data = df_one_hot_encoded, columns = ["Credit History"], prefix = "CreditHistory")
df_one_hot_encoded = pd.get_dummies(data = df_one_hot_encoded, columns = ["Purpose"], prefix = "Purpose")
df_one_hot_encoded = pd.get_dummies(data = df_one_hot_encoded, columns = ["Savings account/bonds"], prefix = "SavingsAccount/Bonds")
df_one_hot_encoded = pd.get_dummies(data = df_one_hot_encoded, columns = ["Present Employment Since"], prefix = "EmployedSince")
df_one_hot_encoded = pd.get_dummies(data = df_one_hot_encoded, columns = ["Personal status and sex"], prefix = "MaritalStatus/Sex")
df_one_hot_encoded = pd.get_dummies(data = df_one_hot_encoded, columns = ["Other debtors/guarantors"], prefix = "OtherDebtors/Guarantors")
df_one_hot_encoded = pd.get_dummies(data = df_one_hot_encoded, columns = ["Property"], prefix = "Property")
df_one_hot_encoded = pd.get_dummies(data = df_one_hot_encoded, columns = ["Other installment plans"], prefix = "OtherInstallmentPlans")
df_one_hot_encoded = pd.get_dummies(data = df_one_hot_encoded, columns = ["Housing"], prefix = "Housing")
df_one_hot_encoded = pd.get_dummies(data = df_one_hot_encoded, columns = ["Job"], prefix = "Job")
df_one_hot_encoded

,Duration in month,Credit_amount,Installment rate in percentage of disposable income,Present residence since,Age in years,Number of existing credits at this bank,Number of people being liable to provide maintenance for,Telephone,Foreign worker,Cost Matrix,...,OtherInstallmentPlans_A141,OtherInstallmentPlans_A142,OtherInstallmentPlans_A143,Housing_A151,Housing_A152,Housing_A153,Job_A171,Job_A172,Job_A173,Job_A174
0,6,1169,4,4,67,2,1,A192,A201,1,...,0,0,1,0,1,0,0,0,1,0
1,48,5951,2,2,22,1,1,A191,A201,2,...,0,0,1,0,1,0,0,0,1,0
2,12,2096,2,3,49,1,2,A191,A201,1,...,0,0,1,0,1,0,0,1,0,0
3,42,7882,2,4,45,1,2,A191,A201,1,...,0,0,1,0,0,1,0,0,1,0
4,24,4870,3,4,53,2,2,A191,A201,2,...,0,0,1,0,0,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,12,1736,3,4,31,1,1,A191,A201,1,...,0,0,1,0,1,0,0,1,0,0
996,30,3857,4,4,40,1,1,A192,A201,1,...,0,0,1,0,1,0,0,0,0,1
997,12,804,4,4,38,1,1,A191,A201,1,...,0,0,1,0,1,0,0,0,1,0
998,45,1845,4,4,23,1,1,A192,A201,2,...,0,0,1,0,0,1,0,0,1,0


In [17]:
df_one_hot_encoded['Telephone'] = df_one_hot_encoded['Telephone'].replace(['A191', 'A192'], [0, 1])
df_one_hot_encoded['Foreign worker'] = df_one_hot_encoded['Foreign worker'].replace(['A201', 'A202'], [1, 0])
df_one_hot_encoded.rename(columns = {'Telephone':'hasPhone', 'Foreign worker':'isForeignWorker', 'Duration in month':'Duration(months)',
                                    'Credit_amount':'CreditAmount', 'Installment rate in percentage of disposable income':'InstallmentRate',
                                    'Present residence since':'ResidentSince','Age in years':'Age(years)',
                                    'Number of existing credits at this bank':'No_ExistingCredits', 
                                    'Number of people being liable to provide maintenance for':'No_Liabilities', 'Cost Matrix':'CostMatrix'}, inplace = True)
df_one_hot_encoded

,Duration(months),CreditAmount,InstallmentRate,ResidentSince,Age(years),No_ExistingCredits,No_Liabilities,hasPhone,isForeignWorker,CostMatrix,...,OtherInstallmentPlans_A141,OtherInstallmentPlans_A142,OtherInstallmentPlans_A143,Housing_A151,Housing_A152,Housing_A153,Job_A171,Job_A172,Job_A173,Job_A174
0,6,1169,4,4,67,2,1,1,1,1,...,0,0,1,0,1,0,0,0,1,0
1,48,5951,2,2,22,1,1,0,1,2,...,0,0,1,0,1,0,0,0,1,0
2,12,2096,2,3,49,1,2,0,1,1,...,0,0,1,0,1,0,0,1,0,0
3,42,7882,2,4,45,1,2,0,1,1,...,0,0,1,0,0,1,0,0,1,0
4,24,4870,3,4,53,2,2,0,1,2,...,0,0,1,0,0,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,12,1736,3,4,31,1,1,0,1,1,...,0,0,1,0,1,0,0,1,0,0
996,30,3857,4,4,40,1,1,1,1,1,...,0,0,1,0,1,0,0,0,0,1
997,12,804,4,4,38,1,1,0,1,1,...,0,0,1,0,1,0,0,0,1,0
998,45,1845,4,4,23,1,1,1,1,2,...,0,0,1,0,0,1,0,0,1,0


In [18]:
df_one_hot_encoded.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 60 columns):
 #   Column                        Non-Null Count  Dtype
---  ------                        --------------  -----
 0   Duration(months)              1000 non-null   int64
 1   CreditAmount                  1000 non-null   int64
 2   InstallmentRate               1000 non-null   int64
 3   ResidentSince                 1000 non-null   int64
 4   Age(years)                    1000 non-null   int64
 5   No_ExistingCredits            1000 non-null   int64
 6   No_Liabilities                1000 non-null   int64
 7   hasPhone                      1000 non-null   int64
 8   isForeignWorker               1000 non-null   int64
 9   CostMatrix                    1000 non-null   int64
 10  CheckingAccount_A11           1000 non-null   uint8
 11  CheckingAccount_A12           1000 non-null   uint8
 12  CheckingAccount_A13           1000 non-null   uint8
 13  CheckingAccount_A14           1000